# Worksheet 08

Name:  Jason Wang  
UID: U73827229  

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [21]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[6.114655801857683, 3.8746181777867523, 6.9351878361205905, 5.369781536192944, 5.2528584115496795, 4.914620939315725, 5.445375698404781, 5.3206358280263775, 3.9571847186256477, 5.886786104991123]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [22]:
s2 = np.random.normal( 8 , 1 , 10 ).tolist()
print(s2)

[8.684569517874694, 8.408753319945827, 6.743318138241413, 6.409451244184588, 7.364048147642151, 7.782942748689725, 6.559437909352123, 9.182050884714602, 9.120890434401053, 7.7375478452637125]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [23]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[5.886786104991123, 7.7375478452637125, 3.9571847186256477, 9.120890434401053, 9.182050884714602, 6.559437909352123, 5.3206358280263775, 7.782942748689725, 7.364048147642151, 5.445375698404781]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

The mean and variance of each component.

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [24]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[7.7375478452637125, 9.120890434401053, 9.182050884714602, 7.782942748689725, 7.364048147642151]
[5.886786104991123, 3.9571847186256477, 6.559437909352123, 5.3206358280263775, 5.445375698404781]
P(S_1) = 0.5,  P(S_2) = 0.5
mean_1 = 8.237496012142248,  mean_2 = 5.433884051880011
var_1 = 0.5784095246916942,  var_2 = 0.7331179789524693


/Users/jasonwang/anaconda3/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [25]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( prob_s[0] * pdf_i[0] / prob_x )
    prob_s1_x.append( prob_s[0] * pdf_i[1] / prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  5.886786104991123
probability of observing that point if it came from cluster 0 =  0.00017868430105358844
probability of observing that point if it came from cluster 1 =  0.4496377812301882
point =  7.7375478452637125
probability of observing that point if it came from cluster 0 =  0.47472684885140326
probability of observing that point if it came from cluster 1 =  0.0039051482131735016
point =  3.9571847186256477
probability of observing that point if it came from cluster 0 =  8.856103988826855e-13
probability of observing that point if it came from cluster 1 =  0.07156589762028366
point =  9.120890434401053
probability of observing that point if it came from cluster 0 =  0.2148617549870863
probability of observing that point if it came from cluster 1 =  1.7515998382680308e-06
point =  9.182050884714602
probability of observing that point if it came from cluster 0 =  0.1818007254568917
probability of observing that point if it came from cluster 1 =  1.1473868305728978e-06
poi

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [30]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x)]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]
var = [sum(map(lambda x : (x - mean[0])**2, prob_s0_x)) / len(prob_s0_x) ,  sum(map(lambda x : (x - mean[0])**2, prob_s1_x)) / len(prob_s1_x)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.5,  P(S_2) = 0.5
mean_1 = 8.231788095235574,  mean_2 = 5.455095813271867
var_1 = 60.06015674425775,  var_2 = 59.97381484674211


g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [32]:
prob_s0_x.append( prob_s[0] * pdf_i[0] / prob_x )
prob_s1_x.append( prob_s[0] * pdf_i[1] / prob_x )

prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x)]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]
var = [sum(map(lambda x : (x - mean[0])**2, prob_s0_x)) / len(prob_s0_x) ,  sum(map(lambda x : (x - mean[0])**2, prob_s1_x)) / len(prob_s1_x)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.5,  P(S_2) = 0.5
mean_1 = 8.231751539362934,  mean_2 = 3.9027006774191433
var_1 = 61.34325132198736,  var_2 = 58.69410647402275


Higher variance in the clusters, and the probability of each point belonging to each cluster is more evenly distributed.

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [35]:
prob_s0_x  =[1]
prob_s1_x = [2]

prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x)]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]
var = [sum(map(lambda x : (x - mean[0])**2, prob_s0_x)) / len(prob_s0_x) ,  sum(map(lambda x : (x - mean[0])**2, prob_s1_x)) / len(prob_s1_x)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.5,  P(S_2) = 0.5
mean_1 = 5.886786104991123,  mean_2 = 5.886786104991123
var_1 = 23.880678435934314,  var_2 = 15.107106225952066
